In [ ]:
!pip install transformers

In [ ]:
!pip install xformers

In [ ]:
!pip install ray[serve]

In [ ]:
!pip install gradio

In [13]:
!pip install -q pyngrok

In [15]:
import ray
import requests
import pandas as pd
import numpy as np
import gradio as gr
import matplotlib.pyplot as plt

from typing import List
from ray import serve
from ray.serve.gradio_integrations import GradioServer
from fastapi import FastAPI
from pydantic import PositiveInt, constr, BaseModel
from pyngrok import ngrok

from transformers import pipeline
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig

https://github.com/deeppavlov/dl-frameworks-course/blob/dev/Pytorch_MLOps/Ray_Serve_NLP.ipynb

In [6]:
ray.init()

2023-06-16 17:10:16,561	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.12
Ray version:,2.5.0
Dashboard:,http://127.0.0.1:8265


In [7]:
serve.start()

In [2]:
# https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
sentiment_latest = AutoModelForSequenceClassification.from_pretrained(MODEL)
model = pipeline("sentiment-analysis", model=sentiment_latest, tokenizer=tokenizer)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
class Item(BaseModel):
    lines: List[str]
    char_names: List[str]
    movie_name: List[str]
    genres: List[str]


app = FastAPI()

@serve.deployment
@serve.ingress(app)
class SentimentAnalyzer:
    """
    A class for analyzing the sentiment of text and dataframes using a pre-trained model.
    """
    def __init__(self):
        self.model = pipeline("sentiment-analysis", model=sentiment_latest, tokenizer=tokenizer)

    @app.post("/df/")
    def analyze_sentiment_df(
        self,
        df_param: List[Item],
        max_lines: PositiveInt = 15,
    ):
        """
        Analyzes the sentiment of a dataframe (json) using a pre-trained model.
        """
        if len(df_param) > max_lines:
          df_param = df_param[:max_lines]
        l = [r.lines for r in df_param][0]
        sent_list = self.model(l)
        label = [r['label'] for r in sent_list]
        return label

    @app.get("/")
    def analyze_sentiment_text(
        self,
        text: constr(min_length=1, strip_whitespace=True),
        max_length: PositiveInt = 300,
    ):
        """
        Analyzes the sentiment of a text string using a pre-trained model.
        """
        if len(text) > max_length:
          text = text[:max_length]
        sent_list = self.model(text)
        sentiment = sent_list[0]["label"]
        return sentiment

SentimentAnalyzer.deploy()

(ServeController pid=13937) INFO 2023-06-16 17:11:22,564 controller 13937 deployment_state.py:1298 - Deploying new version of deployment SentimentAnalyzer.
(ServeController pid=13937) INFO 2023-06-16 17:11:22,612 controller 13937 deployment_state.py:1537 - Adding 1 replica to deployment SentimentAnalyzer.
(ServeController pid=13937) WARNING 2023-06-16 17:11:53,366 controller 13937 deployment_state.py:1849 - Deployment "SentimentAnalyzer" has 1 replicas that have taken more than 30s to be scheduled. This may be caused by waiting for the cluster to auto-scale, or waiting for a runtime environment to install. Resources required for each replica: {"CPU": 1}, resources available: {"CPU": 1.0}.
(ServeReplica:SentimentAnalyzer pid=14280) 2023-06-16 17:11:53.429465: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [15]:
response = requests.post(
    "http://127.0.0.1:8000/SentimentAnalyzer/df/", json=[{"lines": ['Are we supposed to be up here? The rules said...', "Homer you the only one who's read them rules so you the only one who feels like he's doin' somethin' wrong."],
                                                          "char_names": ['HOMER', 'MR. ROSE'],
                                                          "movie_name": ['the cider house rules'],
                                                          "genres": ['drama', 'romance']}]
).text

response

'["neutral","negative"]'

In [16]:
response = requests.get(
    "http://127.0.0.1:8000/SentimentAnalyzer/", params={"text":'I like you. I love you.'}
).text

response

(ServeReplica:SentimentAnalyzer pid=14280) INFO 2023-06-16 17:16:46,060 SentimentAnalyzer SentimentAnalyzer#bHMwQQ XtpTOrmexy /SentimentAnalyzer/ replica.py:654 - __CALL__ OK 167.1ms


'"positive"'

In [18]:
ngrok.connect(8265)

<NgrokTunnel: "https://4804-34-125-143-75.ngrok.io" -> "http://localhost:8265">

In [19]:
serve.shutdown()

(ServeController pid=13937) INFO 2023-06-16 17:17:27,982 controller 13937 deployment_state.py:1264 - Deleting deployment SentimentAnalyzer.
(ServeController pid=13937) INFO 2023-06-16 17:17:28,053 controller 13937 deployment_state.py:1563 - Removing 1 replica from deployment 'SentimentAnalyzer'.


In [20]:
ray.shutdown()

### Gradio


In [3]:
def sent_per_char(str_dict):
    dict_data = eval(str_dict)
    resp = {}
    for i, line in enumerate(dict_data['lines']):
        character = dict_data['char_names'][i]
        labels = model(line)
        label = labels[0]['label']
        if character not in resp:
            resp[character] = {'positive': 0, 'neutral': 0, 'negative': 0}
        if label not in resp[character]:
            resp[character][label] = 0
        resp[character][label] += 1

    fig, axs = plt.subplots(1, len(resp), figsize=(10, 5))
    for i, (p, data) in enumerate(resp.items()):
        axs[i].pie(data.values())
        axs[i].set_xlabel(p)
        axs[i].legend(labels=data.keys(), loc='best')
    plt.tight_layout()
    return fig

In [4]:
test={'lines': ['You are so smart!',
                'Cannot say the same about you.',
                'That was rude!',
                'Whats up guys?'],
           'char_names': ['Bob',
                          'Dan',
                          'Bob',
                          'Rose']
           }

In [5]:
def gradio_sentiment():
    return gr.Interface(
        fn=sent_per_char,
        inputs=[gr.Textbox(label='Dialog format: {"lines":["...", ...], "char_names": ["...", ...]}', value=test)],
        outputs=[gr.Plot()],
    )

In [7]:
gradio_sentiment().launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [8]:
ray.init()

2023-06-16 18:06:35,448	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.12
Ray version:,2.5.0
Dashboard:,http://127.0.0.1:8265


In [9]:
serve.run(
    GradioServer.options(ray_actor_options={"num_cpus": 1}).bind(
        gradio_sentiment
    )
)

(HTTPProxyActor pid=28491) INFO:     Started server process [28491]
(ServeController pid=28446) INFO 2023-06-16 18:06:51,973 controller 28446 deployment_state.py:1298 - Deploying new version of deployment default_GradioIngress.
(ServeController pid=28446) INFO 2023-06-16 18:06:51,977 controller 28446 deployment_state.py:1537 - Adding 1 replica to deployment default_GradioIngress.
(ServeController pid=28446) WARNING 2023-06-16 18:07:22,281 controller 28446 deployment_state.py:1849 - Deployment "default_GradioIngress" has 1 replicas that have taken more than 30s to be scheduled. This may be caused by waiting for the cluster to auto-scale, or waiting for a runtime environment to install. Resources required for each replica: {"CPU": 1.0}, resources available: {"CPU": 1.0}.
(ServeReplica:default_GradioIngress pid=28583) 2023-06-16 18:07:32.382864: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


RayServeSyncHandle(deployment='default_GradioIngress')

In [16]:
ngrok.connect(8265)

<NgrokTunnel: "https://c64d-34-125-143-75.ngrok.io" -> "http://localhost:8265">

In [17]:
serve.shutdown()

(ServeController pid=28446) INFO 2023-06-16 18:10:00,342 controller 28446 deployment_state.py:1264 - Deleting deployment default_GradioIngress.
(ServeController pid=28446) INFO 2023-06-16 18:10:00,374 controller 28446 deployment_state.py:1563 - Removing 1 replica from deployment 'default_GradioIngress'.


In [18]:
ray.shutdown()